In [115]:
import numpy as np
import os
import torch

In [117]:
dir = "C:/Users/evani/OneDrive/AI leiden/Sanquin/RL_matching-main/NN training data/1_1/states"

In [118]:
array_list = []
for file in os.listdir(dir):
    f = os.path.join(dir, file)
    states = np.load(f, allow_pickle=True)
    array_list.append(states)

full = np.vstack(array_list)

In [119]:
len(full)

1143224

In [40]:
full[:, 3]

array([63., 63., 63., ..., 50., 50., 50.])

In [66]:
for c in range(full.shape[1]):
    first = np.percentile(full[:, c], 1)
    nn = np.percentile(full[:, c], 99)
    outliers = []
    for x in full[:, c]:
        if x < first or x > nn:
            outliers.append(x)

    print('column: ', c)
    print('1st: ', first)
    print('99th: ', nn)
    print('# outliers: ', len(outliers))

column:  0
1st:  0.0
99th:  30.0
# outliers:  8318
column:  1
1st:  0.0
99th:  7.0
# outliers:  8244
column:  2
1st:  0.0
99th:  1.0
# outliers:  0
column:  3
1st:  3.0
99th:  65.0
# outliers:  19371
column:  4
1st:  0.0
99th:  45.0
# outliers:  11088
column:  5
1st:  0.0
99th:  1.0
# outliers:  0
column:  6
1st:  0.0
99th:  6.0
# outliers:  6594
column:  7
1st:  0.0
99th:  5.0
# outliers:  10012
column:  8
1st:  0.0
99th:  1.0
# outliers:  0
column:  9
1st:  0.0
99th:  16.0
# outliers:  7718
column:  10
1st:  0.0
99th:  16.0
# outliers:  8916
column:  11
1st:  0.0
99th:  1.0
# outliers:  0
column:  12
1st:  0.0
99th:  18.0
# outliers:  6174
column:  13
1st:  0.0
99th:  15.0
# outliers:  10857
column:  14
1st:  0.0
99th:  1.0
# outliers:  0
column:  15
1st:  9.0
99th:  59.0
# outliers:  19942
column:  16
1st:  0.0
99th:  58.0
# outliers:  11391
column:  17
1st:  0.0
99th:  1.0
# outliers:  0
column:  18
1st:  0.0
99th:  3.0
# outliers:  7103
column:  19
1st:  0.0
99th:  4.0
# outliers:

In [78]:
full[np.where(full[:, 18] > 5)]

array([[28.,  6.,  1., ...,  3.,  7.,  0.],
       [27.,  5.,  1., ...,  3.,  7.,  0.],
       [26.,  4.,  1., ...,  3.,  7.,  0.],
       ...,
       [ 6.,  0.,  0., ...,  3.,  2.,  0.],
       [ 6.,  0.,  0., ...,  2.,  1.,  1.],
       [ 6.,  0.,  0., ...,  2.,  0.,  1.]])

In [90]:
len(np.where(full[:, 3] < 3)[0])

9726

In [51]:
y = np.nonzero(matrix == 1)[1]

In [70]:
np.unique(y, return_counts=True)[1] / len(y) * 100

array([12.61694996, 36.53002386,  2.1517218 ,  6.16659552,  8.85530745,
       32.70347718,  0.34498926,  0.63093497])

In [54]:
from sklearn.model_selection import train_test_split

subset_indices, _ = train_test_split(
    range(len(y)),
    stratify=y,
    train_size=0.1,
)

# Split 80/20 for training and validation
train_set, val_set = train_test_split(
    subset_indices,
    stratify=y[subset_indices],
    test_size=0.2,
)

In [59]:
train = y[train_set]

In [68]:
np.unique(train, return_counts=True)[1] / len(train) * 100

array([12.6168582 , 36.52973529,  2.15183092,  6.1668325 ,  8.85552773,
       32.70389363,  0.34442416,  0.63089758])

In [36]:
from sklearn.model_selection import train_test_split

y = (matrix == 1).nonzero()[:, 1]

subset_indices = train_test_split(
    range(len(matrix)),
    stratify=y,
    train_size=0.1,
)

In [98]:
feature_indices = []
for i in range(24):
    if i % 3 == 0:
        feature_indices.append(i)
        feature_indices.append(i + 1)

In [106]:
import torch

tensor = torch.randint(0, 100, (5,))

In [113]:
tensor[3,]

tensor(22)

In [7]:
def normalize(matrix):
    columns = matrix.shape[1]
    feature_indices = [(i, i + 1) for i in range(columns) if (i % 3 == 0)]
    min_max = []

    for i in feature_indices:
        min_max.append((np.min(matrix[:, i[0]]), np.max(matrix[:, i[0]])))
        min_max.append((np.min(matrix[:, i[1]]), np.max(matrix[:, i[1]])))

    for i in range(len(matrix)):
        index = 0
        for j in feature_indices:
            matrix[i, j[0]] = (
                    (matrix[i, j[0]] - min_max[index][0]) / (min_max[index][1] - min_max[index][0]))
            matrix[i, j[1]] = (
                    (matrix[i, j[1]] - min_max[index + 1][0]) / (min_max[index + 1][1] - min_max[index + 1][0]))
            index += 2
    return matrix

In [8]:
normalize(matrix)

array([[0.45454545, 0.11538462, 1.        , ..., 0.18181818, 0.        ,
        0.        ],
       [0.43181818, 0.07692308, 1.        , ..., 0.18181818, 0.        ,
        0.        ],
       [0.40909091, 0.03846154, 1.        , ..., 0.18181818, 0.        ,
        0.        ],
       ...,
       [0.43181818, 0.        , 0.        , ..., 0.09090909, 0.11764706,
        1.        ],
       [0.43181818, 0.        , 0.        , ..., 0.09090909, 0.05882353,
        1.        ],
       [0.43181818, 0.        , 0.        , ..., 0.09090909, 0.        ,
        1.        ]])

In [51]:
targets = "C:/Users/evani/OneDrive/AI leiden/Sanquin/RL_matching-main/NN training data/1_1/q_matrices"

In [73]:
array_list = []
for file in os.listdir(targets):
    f = os.path.join(targets, file)
    states = np.load(f, allow_pickle=True)
    array_list.append(states)

y = np.vstack(array_list)
y = torch.from_numpy(y).long()

In [91]:
def weighted_sampler(y):
    class_counts = torch.sum(y, dim=0)
    class_weights = 1. / class_counts
    class_weights_all = class_weights[y]

    # weighted_sampler = WeightedRandomSampler(
    #     weights=class_weights_all,
    #     num_samples=len(class_weights_all),
    #     replacement=True
    # )

    # return weighted_sampler

    return class_weights_all, class_weights

In [94]:
torch.sum(y, dim=0)

tensor([144240, 417620,  24599,  70498, 101236, 373874,   3944,   7213])

In [95]:
weights, counts = weighted_sampler(y)

tensor([[1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


In [96]:
import torch.nn as nn

In [143]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)

In [152]:
target = torch.empty(3, dtype=torch.long).random_(5)

In [153]:
target

tensor([0, 4, 4])

In [112]:
indices = (y == 1).nonzero()[:, 1]

In [187]:
from torch.utils.data import Dataset

In [209]:
class MyData(Dataset):
    def __init__(self, data_path=None, target_path=None):
        train_list = []
        for i in os.listdir(data_path):
            data = np.load(data_path + i, allow_pickle=True)
            train_list.append(data)

        test_list = []
        for i in os.listdir(target_path):
            data = np.load(target_path + i, allow_pickle=True)
            test_list.append(data)

        self.x = torch.from_numpy(np.vstack(train_list)).float()
        self.y = torch.from_numpy(np.vstack(test_list)).float()
        self.y = (self.y == 1).nonzero()[:, 1]

    def __getitem__(self, idx):
        sample = self.x[idx], self.y[idx]
        return sample

    def __len__(self):
        return len(self.x)

In [211]:
data_path = 'NN training data/1_1/states/'
target_path = 'NN training data/1_1/q_matrices/'

dataset = MyData(data_path, target_path)

In [222]:
targets = dataset.y

In [232]:
torch.unique(targets, return_counts=True)[1]

tensor([144240, 417620,  24599,  70498, 101236, 373874,   3944,   7213])

In [224]:
class_counts = torch.unique(targets, return_counts=True)[1]
class_weights = 1. / class_counts
class_weights_all = class_weights[targets]

In [247]:
class_counts

tensor([144240, 417620,  24599,  70498, 101236, 373874,   3944,   7213])

In [230]:
1. / class_counts

tensor([6.9329e-06, 2.3945e-06, 4.0652e-05, 1.4185e-05, 9.8779e-06, 2.6747e-06,
        2.5355e-04, 1.3864e-04])

In [227]:
class_weights

tensor([6.9329e-06, 2.3945e-06, 4.0652e-05, 1.4185e-05, 9.8779e-06, 2.6747e-06,
        2.5355e-04, 1.3864e-04])

In [234]:
y_pred = torch.rand(7)

In [243]:
y_pred = torch.tensor([[0.1453, -0.0590, -0.0065, 0.0905, 0.0146, -0.0805, -0.1211, -0.0394,
                        -0.0181, -0.0136]])

In [249]:
torch.max(torch.log_softmax(y_pred, dim=1), dim=1)

torch.return_types.max(
values=tensor([-2.1513]),
indices=tensor([0]))

In [250]:
loss = loss = nn.CrossEntropyLoss()

In [263]:
target = torch.tensor([0])

In [280]:
y_pred = torch.tensor([[0.6, 0.4, 0.3, 0.22, 0.55]])

In [281]:
loss(y_pred, target)

tensor(1.4338)